PART 1

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

In [12]:
data_transforms = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [13]:
trainset = datasets.CIFAR100(root='.', train = True, transform = data_transforms,
download=True)
testset = datasets.CIFAR100(root='.', train=False, transform = data_transforms,
download=True)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
data = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)


In [15]:
model = models.vgg16(pretrained = True)
print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
num_in_ftrs = model.classifier[6].in_features
print(num_in_ftrs)


4096


In [17]:
num_cls = 100
model.classifier[6] = nn.Linear(num_in_ftrs, num_cls, bias=True)

In [18]:
for param in model.parameters():
  param.requires_grad = False
for param in model.classifier[6].parameters():
  param.requires_grad = True

In [19]:
num_epochs = 10

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
for i in range(10):
  for j, batch in enumerate(data):
    images, labels = batch
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    outputs = model(images)

    loss = criterion(outputs,labels)
    loss.backward()

    optimizer.step()


  scheduler.step()

  best_model_str = 'best_model' + str(i) + '.pth'
  torch.save(model.state_dict(), best_model_str)



In [21]:
data = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [24]:
accuracy = 0
for i in range(10):
  model.load_state_dict(torch.load('best_model' + str(i) + '.pth'))
  model.eval()
  predictions_array = []
  labels_array = []
  for i in range(1):
    for j, batch in enumerate(data):

      images, labels = batch
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predictions = torch.max(outputs, 1)
      predictions_array += predictions
      labels_array += labels

  correlation_count = 0
  samples = len(predictions_array)
  for i in range(len(predictions_array)):
    if predictions_array[i].item() == labels_array[i].item():
      correlation_count += 1

  temporary_accuracy = correlation_count/samples

  if temporary_accuracy > accuracy:
    accuracy = temporary_accuracy
    print(accuracy)
    print(i)

0.5942
9999
0.597
9999
0.5994
9999
0.6002
9999
0.6012
9999
0.6018
9999
